The idea is to create a json file of natural language questions from the url of the most run queries in a given Explore/Model.
This can then be uploaded to the examples table in BigQuery.

In [ ]:
pip install looker_sdk

In [ ]:
project_id = 'GCP_PROJECT_ID'
location = 'GCP_LOCATION' # us-central1

In [ ]:
!gcloud auth application-default login

In [ ]:
!gcloud config set project {project_id}
!gcloud auth application-default set-quota-project {project_id}

In [ ]:
import google
from google.auth import credentials
from google.auth.transport.requests import Request

# Assuming gcloud authentication is done, you can now use google-auth to get credentials.
credentials, project = google.auth.default()

if credentials.expired:
    credentials.refresh(Request())

print(f"Authenticated to project: {project}")


In [ ]:
import looker_sdk
import json
import os
import vertexai
from vertexai.preview.generative_models import GenerativeModel, GenerationConfig
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

os.environ['LOOKERSDK_BASE_URL'] = 'LOOKERSDK_BASE_URL' #https://instancename.looker.app
os.environ['LOOKERSDK_CLIENT_ID'] = 'LOOKERSDK_CLIENT_ID'
os.environ['LOOKERSDK_CLIENT_SECRET'] = 'LOOKERSDK_CLIENT_SECRET'
sdk = looker_sdk.init40()

In [ ]:
num_of_examples = 20

hostname = "LOOKER_HOSTNAME" #https://instancename.looker.app
model = "LOOKER_MODEL" #fashionly
explore = "LOOKER_EXPLORE" #order_items

# Enter here any history slugs that you want in the examples.
# The rest will be generated from the most queried queries.

history_slugs = [
    '2ec605495cfc7f1f82a97d71217c203d',
    'dfded515ee5bf66a81323f4ef4990863'
]



In [ ]:
fields = [
    "query.view",
    "query.id",
    "query.fields",
    "query.filters",
    "query.limit",
    "query.column_limit",
    "query.sorts",
    "query.pivots_used",
    "history.count"
]

def get_fields(num_of_examples, model, explore, history_slugs):
  body_queries = {
      "model":"system__activity",
      "view":"history",
      "fields": fields,
      "filters":{"history.slug": "'" + "','".join(history_slugs) + "'"},
      "query_timezone":"UTC",
      "sorts": ["query.limit desc", "history.count desc"],
      "limit": max(1, num_of_examples-len(history_slugs))
    }
  results_queries = sdk.run_inline_query(result_format='json', body=body_queries)
  results_queries_json = json.loads(results_queries)

  body = {
      "model":"system__activity",
      "view":"history",
      "fields": fields,
      "filters":{"query.model":model, "query.view":explore},
      "query_timezone":"UTC",
      "sorts": ["query.limit desc", "history.count desc"],
      "limit": max(5, num_of_examples-len(history_slugs))
    }
  results = sdk.run_inline_query(result_format='json', body=body)
  results_json = json.loads(results)

  merged_json = results_json + results_queries_json

  print(merged_json)
  return merged_json

In [ ]:
modified_urls = []
modified_fields = []

results_json = get_fields(num_of_examples, model, explore, history_slugs)

for i in results_json:
  # Original URL
  base_url = f"{hostname}/explore/{model}"

  # Dynamic parameters to modify
  try:
    fields = i.get('query.fields').strip('[]').replace('"', '')
  except:
    fields = ""

  try:
    filters_key_value_pairs = i.get('query.filters').replace('"', '').strip("{}").split(",")
    filters = "&".join(f"f[{pair.split(':')[0]}]={pair.split(':')[1]}" for pair in filters_key_value_pairs).replace(' ', '+')
  except:
    filters = ""

  try:
    sorts = "&".join(json.loads(i.get('query.sorts').replace(' ', '+')))
  except:
    sorts = ""

  limit = i.get('query.limit')
  column_limit = i.get('query.column_limit')

  # Construct the modified URL
  modified_base_url = (
      f"{base_url}/" # base
      f"{i.get('query.view')}?" # explore
  )

  fields_url = (
      f"fields={fields}&" # fields
      f"{filters}&"
      f"sorts={sorts}&"
      f"limit={limit}&"
      f"column_limit={column_limit}&"
      f"origin=share-expanded"
  )

  modified_url = modified_base_url+fields_url

  print("Modified URL:", modified_url)
  modified_urls.append(modified_url)
  modified_fields.append(fields_url)

print(modified_urls)
print(modified_fields)


In [ ]:
prompt = '''
You are a specialized assistant that translates the fields parameter in a Looker Explore query URL into clear, natural language questions.

By analyzing the provided fields and filters in the URL, you will generate a concise, one-sentence question that resembles something an average person would ask, avoiding technical jargon. Keep responses short and conversational.

Example Input:
fields=users.count&f[products.category]=Jeans&f[order_items.created_date]=1+years&limit=500&origin=share-expanded

Example Output:
How many users bought jeans last year?

Now, generate the question for this input:
'''

parameters = {"max_output_tokens": 2500, "temperature": 0.2, "candidate_count": 1}
vertexai.init(project=project_id, location=location)

def generate_input(prompt_part):
    model = GenerativeModel("gemini-pro")
    response =  model.generate_content(
        contents=prompt + prompt_part,
        generation_config=GenerationConfig(
            temperature=0.2,
            top_p=0.8,
            top_k=40,
            max_output_tokens=1000,
            candidate_count=1
        )
    )
    print(response.text)
    return response.text

In [ ]:
responses = []
for field in modified_fields:
  responses.append({'input': generate_input(field).strip(),'output': field})

In [ ]:
import pprint as pp
for i in responses:
  print('---')
  print(i.get('input'))
  print(i.get('output'))

In [ ]:
with open("examples.json", "w") as file:
    json.dump(responses, file, indent=2)